# 看穿你的意圖(keras版)

In [1]:
import codecs
import numpy as np
import random
import pickle
import os
import math

In [2]:
intent_data=codecs.open('intent_list.txt',mode='r',encoding='utf-8-sig').readlines()
intent_dict={}
for i in range(len(intent_data)):
    intent_dict[i]=intent_data[i].strip() 
intent_dict

{0: '打開app應用',
 1: '詢問公車路線',
 2: '數學計算',
 3: '閒聊',
 4: '詢問電影場次',
 5: '查詢聯絡人',
 6: '查詢食譜',
 7: '詢問日期',
 8: '電子郵件指令',
 9: '詢問電視節目',
 10: '詢問航班',
 11: '詢問健康資訊',
 12: '詢問樂透',
 13: '詢問地圖',
 14: '詢問足球賽事',
 15: '簡訊指令',
 16: '音樂指令',
 17: '詢問新聞',
 18: '詢問小說',
 19: '詢問詩詞',
 20: '詢問廣播節目',
 21: '詢問謎語',
 22: '鬧鐘時程設定',
 23: '詢問股票',
 24: '電話指令',
 25: '詢問火車班次',
 26: '詢問翻譯',
 27: '切換電視頻道',
 28: '影片播放指令',
 29: '詢問天氣',
 30: '開啟網站'}

In [3]:
train_data=codecs.open('../Data/ex13_train/Intent_TrainData.txt',mode='r',encoding='utf-8-sig').readlines()
maxlen=max([len(s) for s in train_data])
print("最大長度為:{0}".format(maxlen))
print(len(train_data))
print(train_data[:10])

最大長度為:52
2299


In [4]:
test_data=codecs.open('../Data/ex13_train/Intent_TestData.txt',mode='r',encoding='utf-8-sig').readlines()
maxlen=max([len(s) for s in test_data])
print("最大長度為:{0}".format(maxlen))
print(len(test_data))
print(test_data[:10])

最大長度為:69
1437


In [5]:
chars={}
with open('hanzi2vec256.pkl','rb') as fp:
    chars = pickle.load(fp)

In [6]:
random.shuffle(train_data) #把訓練數據集順序打亂
random.shuffle(test_data) #把訓練數據集順序打亂
idx0=0
idx1=0

def data_augumentation(txt):
    txt=list(txt)
    if random.randint(0,10)<=3 and len(txt)>6:#隨機對調字的順序
        pos= random.randint(1,len(txt)-2)
        w1=txt[pos]
        w2=txt[pos+1]
        txt[pos]=w2
        txt[pos+1]=w1
    if random.randint(0,100)%7<2 and len(txt)>=6:
        wordlist=['的','了','若','可','然','也','或','，','。',' ','  ']
        w1=random.choice(wordlist)
        txt.insert(random.randint(1,len(txt)-2), w1)
    if random.randint(0,100)%5<2:
        wordlist=['，','。','？','！',' ','  ']
        w1=random.choice(wordlist)
        txt.append(w1)
    return ''.join(txt)

def get_next_minibatch(minibatch_size=16,is_train=True):
    global idx0,idx1

    features = []
    labels = []
    groundtruths = []
    while len(features) < minibatch_size:
        try:
            items=[]
            if is_train:
                items=train_data[idx0].strip().split('\t')
            else:
                items=test_data[idx1].strip().split('\t')
            txt=items[0]
            new_txt=items[0]
            if is_train:
                new_txt=data_augumentation(new_txt)
            lab=int(items[1])
            feature=np.zeros((256,128)) #預設都為零
            label=np.zeros(31)
            label[lab]=1
            for i in range(128):
                if i< len(new_txt):
                    feature[:,i]=chars[new_txt[i].lower()] if new_txt[i].lower() in chars else np.random.uniform(-1,1,256)
                    #替換為字向量，若是沒有這個字，填隨機亂數
            features.append(feature.transpose())
            labels.append(label)
            groundtruths.append(list(txt))
            if is_train:
                idx0 += 1
                if idx0 >= len(train_data):
                    idx0 = 0
                    random.shuffle(train_data)
            else:
                idx1 += 1
                if idx1 >= len(test_data):
                    idx1 = 0
                    random.shuffle(test_data)
        except Exception as e:
            print(e)
    return np.asarray(features).astype(np.float32), np.asarray(labels).astype(np.float32), groundtruths


features,labels,groundtruths=get_next_minibatch(minibatch_size=3)
print(features)
print(labels)
print(groundtruths)

[[[-0.75009847  0.09500695  0.91376895 ...  1.0337592   1.3697404
   -0.59126204]
  [ 0.60782826 -1.3320998  -1.4303396  ... -0.7824813  -0.9995339
   -0.7356023 ]
  [ 0.60782826 -1.3320998  -1.4303396  ... -0.7824813  -0.9995339
   -0.7356023 ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.3429284   0.7419523   0.1245196  ... -0.07776023 -0.6644042
   -0.09694016]
  [-0.59841084  0.6645831   0.3415414  ...  1.1762546  -0.10037854
    0.29701737]
  [-1.3728026   0.94152653  0.47251025 ...  0.39836767  0.5001551
    0.44415528]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.51127464  0.45640758  0.56758267 ..

In [7]:
from keras import backend as K
from keras.callbacks import LambdaCallback
from keras.models import *
from keras.layers import *
from keras.losses import *
from keras.optimizers import RMSprop


Using TensorFlow backend.


![md_images](../Images/textcnn.jpg)

In [ ]:


#為了解決類別間不均衡的焦距損失
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))+K.categorical_crossentropy(y_true, y_pred)
    return focal_loss_fixed


#keras通道在後
model_input = Input(shape=(128,256))
z = model_input
z = Dropout(0.2)(z)
# Convolutional block
conv_blocks = []
filter_sizes=[3,5,7]
for sz in filter_sizes:
    conv = Convolution1D(filters=64,
                         kernel_size=sz,
                         padding="same",
                         activation='relu',
                         strides=2)(z)
    conv_blocks.append(conv)
z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z = Dropout(0.5)(z)
model_output = Convolution1D(filters=64,
                         kernel_size=3,
                         padding="same",
                         activation='relu',
                         strides=1)(z)
model_output = Convolution1D(filters=16,
                         kernel_size=1,
                         padding="same",
                         activation='relu',
                         strides=1)(model_output)
model_output=Flatten()(model_output)
model_output=Dense(31, activation="relu")(model_output)
model = Model(model_input, model_output)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
minibatch_size=32
def on_epoch_end(epoch,logs):
    # Function invoked at end of each epoch. Prints generated text.
    x_pred, x_label, truths = get_next_minibatch(minibatch_size,is_train=False)
    pred=model.predict(x_pred, verbose=0)[0]
    model.save_weights('Models/textcnn_keras.h5',True)
    for i in range(5):
        sentence=truths[i]
        if len(sentence)>=3:
            result=np.argmax(pred[i])
            label=np.argmax(x_label[i])
            print('{0}  ({1})=>({2})'.format(''.join(sentence),intent_dict[label],intent_dict[result]))

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
x,y,truth=get_next_minibatch(len(train_data))
print(len(x))
model.fit(x, y,batch_size=minibatch_size,epochs=300,callbacks=[print_callback])


2299
Epoch 1/300
2299/2299 [==============================] - 11s 5ms/step - loss: 7.8503 - acc: 0.1701
4的平方根的立方加9的2次方根的3次方  (數學計算)=>(打開app應用)
中超赛事预告，你。  (詢問足球賽事)=>(打開app應用)
武汉理工大学在哪  (詢問地圖)=>(打開app應用)
CCTV4中文国际  (切換電視頻道)=>(打開app應用)
搜索大乐透的号码资讯  (詢問樂透)=>(打開app應用)
Epoch 2/300
2299/2299 [==============================] - 1s 648us/step - loss: 6.5095 - acc: 0.2018
查询上海到丽江飞机票的价格  (詢問航班)=>(打開app應用)
你怎么就知道我不会呢？  (閒聊)=>(打開app應用)
姜堰到南京的火车  (詢問火車班次)=>(打開app應用)
前天中央一套的节目单  (詢問電視節目)=>(打開app應用)
郑州的天气怎么样  (詢問天氣)=>(打開app應用)
Epoch 3/300
2299/2299 [==============================] - 2s 768us/step - loss: 6.4147 - acc: 0.1979
糖醋排骨怎么做？好了额，你。  (查詢食譜)=>(打開app應用)
从东莞到厦门的火车票  (詢問火車班次)=>(打開app應用)
中国建设银行的行情  (詢問股票)=>(打開app應用)
那你男神是谁  (閒聊)=>(打開app應用)
你QQ号吗诗多少  (閒聊)=>(打開app應用)
Epoch 4/300
2299/2299 [==============================] - 2s 934us/step - loss: 6.3774 - acc: 0.2088
搜索米聊  (打開app應用)=>(打開app應用)
我要给陈永信息  (簡訊指令)=>(打開app應用)
呼叫王希  (電話指令)=>(打開app應用)
去桂林的航班怎样  (詢問航班)=>(打開app應用)
Epoch 5/300
2299/2299 [===========